# First trajectory-optimization problems
This notebook drives you through simple trajectory optimization problems, considering only "kinematic" version i.e only motion variables (no forces or torques yet). 
We will first look at a problem where the trajectory is described by a sequence of configurations $[q_0 ... q_T]$, with the only constraints is to have them sufficiently closed to each other.
The we will look at a real integration, where each timestep is described by a configuration $q_t$, velocity $v_t$ and acceleration $a_t$ (all in the configuration space. 
This will allow us to formulate a proper contact constraint, even if only partially described from the motion point of view. 
The next notebook will drive us to complete version of the trajectory optimization problem, including forces, torques and complete contacts.



In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will use the following tools:
- the ur10 model (loaded by example-robot-data)
- pinocchio.casadi for writing the problem and computing its derivatives
- the IpOpt solver wrapped in casadi
- the meshcat viewer

In [2]:
# %load tp2/generated/trajopt_kine_imports
import time
import unittest
import numpy as np
import pinocchio as pin
import casadi
from pinocchio import casadi as cpin
import example_robot_data as robex
from scipy.optimize import fmin_bfgs
from numpy.linalg import norm

from utils.meshcat_viewer_wrapper import MeshcatVisualizer


The notebook is written first for a UR10 robot model configured in fixed basis, then with a Talos pair of legs configured in floating mode. Replace the robot by your own URDF...

In [3]:
robot = robex.load('ur10')

The robot will start from an arbitrary initial configuration (make the solver life simpler by avoiding the singular 0 configuration) and aim at an object located a placement $M_{target} \in SE(3)$

In [4]:
# %load tp2/generated/trajopt_kine_configurations
Mtarget = pin.SE3(pin.utils.rotate('y', 3), np.array([-0.5, 0.1, 0.2]))  # x,y,z
q0 = np.array([0, -3.14 / 2, 0, 0, 0, 0])
endEffectorFrameName = 'tool0'


Open a viewer, display the robot and markers to visualize

In [5]:
# %load tp2/generated/trajopt_kine_viewer
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
# Add a vizualization for the target
boxID = "world/box"
viz.addBox(boxID, [.05, .1, .2], [1., .2, .2, .5])
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [.08] * 3, [.2, .2, 1., .5])
def displayScene(q,dt=1e-1):
    '''
    Given the robot configuration, display:
    - the robot
    - a box representing endEffector_ID
    - a box representing Mtarget
    '''
    pin.framesForwardKinematics(model,data,q)
    M = data.oMf[endEffector_ID]
    viz.applyConfiguration(boxID, Mtarget)
    viz.applyConfiguration(tipID, M)
    viz.display(q)
    time.sleep(dt)


Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:
viz.viewer.jupyter_cell()

We are mostly interested by the robot model. We also need to access some particular frames.

In [7]:
# %load tp2/generated/trajopt_kine_modeldata
robot.q0 = q0

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()
endEffector_ID = model.getFrameId(endEffectorFrameName)


Finally, we will need the same helpers for getting around the Casadi SX/MX issue.

In [8]:
# %load tp2/generated/trajopt_kine_casadi
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

cq = casadi.SX.sym("x",model.nq,1)
cpin.framesForwardKinematics(cmodel,cdata,cq)

error3_tool = casadi.Function('etool3', [cq],
                              [ cdata.oMf[endEffector_ID].translation - Mtarget.translation ])
error6_tool = casadi.Function('etool6', [cq],
                             [ cpin.log6(cdata.oMf[endEffector_ID].inverse() * cpin.SE3(Mtarget)).vector ])
error_tool = error3_tool


We are done for the set up, let's noew write the problems we are interested by.

## Optimizing a sequence of configurations
We will first consider a problem where the trajectory is represented by a sequence of configurations.

Decide: $Q = [ q_0 ... q_T ] \in R^{n_q \times (T+1)}$

Minimizing:   $sum_{t=0}^{T-1}  || q_t - q_{t+1} ||^2 + || e(q_T) ||^2$

Subject to: $q_0 = robot.q0$

The hyperparameters for this problems are:

In [9]:
# %load tp2/generated/trajopt_kine_hyper
T = 10
w_run = .1
w_term = 1


For that, we first define an optimization problem in Casadi where the decision variables are the T+1 configurations.

In [10]:
# %load tp2/generated/trajopt_kine_casadi_opti
opti = casadi.Opti()
var_qs = [ opti.variable(model.nq) for t in range(T+1) ]
totalcost = 0


Now implement the running cost (the sum of configuration differences):

In [11]:
# %load tp2/generated/trajopt_kine_casadi_runcost
for t in range(T):
    totalcost += w_run * casadi.sumsqr( var_qs[t] - var_qs[t+1] )


Now implement the terminal cost. You can use either the 3d cost (position only) or 6d cost (full placement).

In [12]:
# %load tp2/generated/trajopt_kine_casadi_termcost
totalcost += w_term * casadi.sumsqr( error_tool(var_qs[T]) )


Now implement the initial constraint.

In [13]:
# %load tp2/generated/trajopt_kine_casadi_q0
opti.subject_to(var_qs[0] == robot.q0)


And we are good for optimizing the problem:

In [14]:
# %load tp2/generated/trajopt_kine_solve
opti.minimize(totalcost)
opti.solver("ipopt") # set numerical backend
opti.callback(lambda i: displayScene(opti.debug.value(var_qs[-1])))

# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_qs = [ opti.value(var_q) for var_q in var_qs ]
except:
    print('ERROR in convergence, plotting debug info.')
    sol_qs = [ opti.debug.value(var_q) for var_q in var_qs ]



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      136

Total number of variables............................:       66
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

Here the progress of the solver is shown by displaying the terminal configuration in the viewer at each new step. We can also visualize the full trajectory.

In [15]:
# %load tp2/generated/trajopt_kine_disptraj
def displayTraj(qs,dt=1e-2):
    for q in qs[1:]:
        displayScene(q,dt=dt)


In [16]:
displayTraj(sol_qs)